In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
from datetime import datetime

import tensorflow as tf
from tensorflow import keras

In [3]:
import tensorboard
tensorboard.__version__

'2.2.1'

In [4]:
# Clear any logs from previous runs
!rm -rf ./logs/ 


In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])


In [6]:
(train_images, train_labels), _ = keras.datasets.fashion_mnist.load_data()
train_images = train_images / 255.0


In [7]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
model.fit(
    train_images,
    train_labels, 
    batch_size=64,
    epochs=5, 
    callbacks=[tensorboard_callback])


Epoch 1/5
938/938 [==============================] - 9s 9ms/step - loss: 0.6991 - accuracy: 0.7574
Epoch 2/5
938/938 [==============================] - 8s 8ms/step - loss: 0.4933 - accuracy: 0.8255
Epoch 3/5
938/938 [==============================] - 8s 8ms/step - loss: 0.4473 - accuracy: 0.8403
Epoch 4/5
938/938 [==============================] - 11s 12ms/step - loss: 0.4278 - accuracy: 0.8468
Epoch 5/5
938/938 [==============================] - 10s 10ms/step - loss: 0.4123 - accuracy: 0.8513


In [8]:
%tensorboard --logdir logs

In [9]:
# The function to be traced.
@tf.function
def my_func(x, y):
  # A simple hand-rolled layer.
  return tf.nn.relu(tf.matmul(x, y))

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)


In [10]:
%tensorboard --logdir logs/func